# Hackathon
# Trabajo de desarrollo y selección de modelos
Autores: 
Equipo ARCA: José Alcayaga, David Araya, Marcelo Céspedes, Ignacia Rivas, Jorge Rivera

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV, ElasticNetCV, SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, cross_validate
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
columns_rename = {
 'tiempo': 'T',
 '% Solido  Bombeo concentrado_EB': 'EB%', 
 'Presión de Descarga_EB_1': 'EB1',
 'Presión de Descarga_EB_2': 'EB2',
 'Presion_Estación de Valvulas_EV1_1': 'EV1_1',
 'Presión estación de valvulas 2_EV2_1': 'EV2_1',
 'Presion_Estación de Valvulas_EV1_2': 'EV1_2',
 'Presión estación de valvulas 2_EV2_2': 'EV2_2',
 'Presión_SM-1': 'SM1',
 'Presión_SM-2': 'SM2',
 'Presión_SM-3': 'SM3',
 'Presión_SM-4': 'SM4',
 'Porcentaje de Solido Alimentación Espesador': 'EDT%',
 'Presión_EDT_1': 'EDT1',
 'Presión_EDT_2': 'EDT2',
 'Presión_EDT_3': 'EDT3'
}

# Lectura de datos

In [3]:
data = pd.read_excel('Hack_concentraducto_v01.xlsx', sheet_name='Data_Hackathon')
df1 = data.copy()
df1.head()

KeyboardInterrupt: 

In [ ]:
df1.isnull().sum()

# Preprocesamiento
## Conversión de datos

In [ ]:
df2 = data.copy()
df2.rename(columns=columns_rename, inplace=True)

### Conversión de presiones

In [ ]:
conversion = 6.89476

df2[['EB1','EB2']] = df2[['EB1','EB2']].apply(lambda x: x * conversion)

### Normalización de datos
El resultado de esta etapa fue utilizado para el análisis exploratorio de datos, pero no se considero esta transformación para los modelos finales, ya que en los modelos escogidos, no generaban diferencias, y se quiso evitar transformaciones innecesesarias entre las medidas (puesto que, si se normalizaban los datos, se debería hacer la transformación inversa para obtener los valores reales).

In [ ]:
excluded_columns = ['T']
normalize_columns = [col for col in df2.columns if col not in excluded_columns]

scaler = StandardScaler()

df_normalized = pd.DataFrame(scaler.fit_transform(df2[normalize_columns]), columns=normalize_columns)
df_normalized = pd.concat([df2[excluded_columns], df_normalized], axis=1)


# Visualización de datos

In [ ]:
pd.plotting.scatter_matrix(df2)

### Correlación entre SM2 y EV2
Analizando el conjunto de datos, se encontró que, en casos donde la presión de SM2 cambia entre dos tuplas de manera drástica, el cambio se propaga de manera instantánea a la presión de EV2. Esto se puede observar en el siguiente gráfico:

In [ ]:
plt.scatter(df_normalized['SM2'], df_normalized['EV1_1'], xlabel='SM2', ylabel='EV1_1')

In [ ]:
plt.scatter(df_normalized['SM4'], df_normalized['EV2_2'])

In [ ]:
plt.scatter(df_normalized['SM4'], df_normalized['EDT1'])

In [ ]:
plt.scatter(df_normalized['SM1'], df_normalized['EB2'])

In [ ]:
plt.scatter(df_normalized['SM1'], df_normalized['SM2'])

In [ ]:
plt.scatter(df_normalized['SM3'], df_normalized['EV2_1'])

In [ ]:
plt.scatter(df_normalized['SM3'], df_normalized['EV1_2'])

# Creación de modelos
A continuación se presentan los modelos creados para cada una de las mediciones de presión.
Para cada estación, se probaron de manera manual una serie de modelos predictivos. En primera instancia, se probraron los modelos lineales:

 -  Regresión Lineal
 -  Regresión Lasso
 -  Elastic Net
 -  Modelo con Descenso por Gradiente Estocástico

En todos los casos, la regresión lineal entregó los mejores resultados, pero no se obtuvo un puntaje satisfactorio.
Por esto, se indagaron modelos no lineales, acción que se justificó al visualizarse cierto comportamiento no lineal entre los datos. Se consideraron los modelos:

 -  Regresión Lineal con Características Polinomiales
 -  Árbol de Decisión Mejorado por Gradiente

Efectivamente se obtuvieron mejores resultados con dichos modelos, obteniéndose las mejores puntuaciones con el árbol de decisión en todos los modelos, menos el de SM2.
Además se barajaron otros dos modelos, SVM y Random Forest; su entrenamiento tomaba un tiempo considerable, mayor sugerido para la competencia, y en ocasiones los equipos usados no podían completar el trabajo. Por esto, simplemente fueron descartados.
Así, se escogieron los siguientes modelos:

| Estación | Modelo     |
|----------|------------|
| SM1      | XGB        |
| SM2      | Polinomial |
| SM3      | XGB        |
| SM4      | XGB        |


## Predicción SM1
Se detectó una alta correlación entre SM1 y los sensores de EB. Además, se consideraron los sensores de EV1 y SM2 para mejorar la presición del modelo.

In [ ]:
X_1 = df2[['EB%','EB1','EB2','SM2','EV1_1']]
y_1 = df2['SM1']
X_train, X_test, y_train, y_test = train_test_split(X_1, y_1, random_state=42, test_size=0.4)

### Regresión Linear

In [ ]:
sm1 = LinearRegression()
%time sm1.fit(X_train, y_train)
y_pred = sm1.predict(X_test)
r2_1 = r2_score(y_test, y_pred)
mae_1 = mean_absolute_error(y_test, y_pred)
rmse_1 = mean_squared_error(y_test, y_pred, squared=False)
print(f'SM1: R2 = {r2_1}, MAE = {mae_1}, RMSE = {rmse_1}')
cross_validate(sm1, X_1, y_1, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

Observaciones:
Mayor puntuación, dentro de modelos lineales, con un 40% de datos asignados para validación, con todos los datos de presión en [kPa].

### Regresión Linear Polinomial

In [ ]:
poly = PolynomialFeatures(degree=3)
linear = LinearRegression()
sm1 = make_pipeline(poly, linear)
%time sm1.fit(X_train, y_train)
y_pred = sm1.predict(X_test)
r2_1 = r2_score(y_test, y_pred)
mae_1 = mean_absolute_error(y_test, y_pred)
rmse_1 = mean_squared_error(y_test, y_pred, squared=False)
print(f'SM1: R2 = {r2_1}, MAE = {mae_1}, RMSE = {rmse_1}')
cross_validate(sm1, X_1, y_1, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Regresión Lasso

In [ ]:
sm1 = LassoCV(max_iter=5000)
sm1.fit(X_train, y_train)
y_pred = sm1.predict(X_test)
r2_1 = r2_score(y_test, y_pred)
mae_1 = mean_absolute_error(y_test, y_pred)
rmse_1 = mean_squared_error(y_test, y_pred, squared=False)
print(f'SM1: R2 = {r2_1}, MAE = {mae_1}, RMSE = {rmse_1}')
cross_validate(sm1, X_1, y_1, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

Observaciones: 
En primer intento, se tuvo menor puntaje y problemas de convergencia.
Se aumenta número de iteraciones y se obtiene mayor demora, manteniendo puntaje.

### Elastic Net

In [ ]:
sm1 = ElasticNetCV()
sm1.fit(X_train, y_train)
y_pred = sm1.predict(X_test)
r2_1 = r2_score(y_test, y_pred)
mae_1 = mean_absolute_error(y_test, y_pred)
rmse_1 = mean_squared_error(y_test, y_pred, squared=False)
print(f'SM1: R2 = {r2_1}, MAE = {mae_1}, RMSE = {rmse_1}')
cross_validate(sm1, X_1, y_1, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### SGD

In [ ]:
sm1 = SGDRegressor()
sm1.fit(X_train, y_train)
y_pred = sm1.predict(X_test)
r2_1 = r2_score(y_test, y_pred)
mae_1 = mean_absolute_error(y_test, y_pred)
rmse_1 = mean_squared_error(y_test, y_pred, squared=False)
print(f'SM1: R2 = {r2_1}, MAE = {mae_1}, RMSE = {rmse_1}')
cross_validate(sm1, X_1, y_1, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

Observaciones: 
Solo funciona con datos estandarizados. Para simplificar la salida del modelo, de forma que no se tenga que pasar por una etapa extra de transformación inversa, y considerando que en la mayoría de los modelos no hubieron mejoras substanciales, se optó simplemente por usar los datos en sus medidas originales.

### XGBoost

In [ ]:
sm1 = XGBRegressor(random_state=42, n_estimators=200, learning_rate=0.3, reg_lambda=0.5, reg_alpha=0.25)
%time sm1.fit(X_train, y_train)
y_pred = sm1.predict(X_test)
r2_1 = r2_score(y_test, y_pred)
mae_1 = mean_absolute_error(y_test, y_pred)
rmse_1 = mean_squared_error(y_test, y_pred, squared=False)
print(f'SM1: R2 = {r2_1}, MAE = {mae_1}, RMSE = {rmse_1}')
cross_validate(sm1, X_1, y_1, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

Observaciones: 
Presenta la mejor puntuación de todos los modelos. Se escoge este.

### Random Forest

Observaciones:
Entrenamiento lento, fuera de plazo máximo. Descartado.

## Predicción SM2
Luego del análisis explorativo, se encontró una alta correlación entre las mediciones de SM2 y EV1, por lo cual se escogen sus sensores como únicas entradas del modelo.

In [ ]:
X_2 = df2[['EV1_1', 'EV1_2']]
y_2 = df2['SM2']
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, random_state=42, test_size=0.4)

### Regresión Linear

In [ ]:
sm2 = LinearRegression()
%time sm2.fit(X_2_train, y_2_train)
y_2_pred = sm2.predict(X_2_test)
r2_2 = r2_score(y_2_test, y_2_pred)
mae_2 = mean_absolute_error(y_2_test, y_2_pred)
rmse_2 = mean_squared_error(y_2_test, y_2_pred, squared=False)
print(f'SM2: R2 = {r2_2}, MAE = {mae_2}, RMSE = {rmse_2}')
cross_validate(sm2, X_2, y_2, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Regresión Linear Polinomial

In [ ]:
poly = PolynomialFeatures(degree=4)
linear = LinearRegression()
sm2 = make_pipeline(poly, linear)
%time sm2.fit(X_2_train, y_2_train)
y_2_pred = sm2.predict(X_2_test)
r2_2 = r2_score(y_2_test, y_2_pred)
mae_2 = mean_absolute_error(y_2_test, y_2_pred)
rmse_2 = mean_squared_error(y_2_test, y_2_pred, squared=False)
print(f'SM2: R2 = {r2_2}, MAE = {mae_2}, RMSE = {rmse_2}')
cross_validate(sm2, X_2, y_2, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Regresión Lasso

In [ ]:
sm2 = LassoCV()
sm2.fit(X_2_train, y_2_train)
y_2_pred = sm2.predict(X_2_test)
r2_2 = r2_score(y_2_test, y_2_pred)
mae_2 = mean_absolute_error(y_2_test, y_2_pred)
rmse_2 = mean_squared_error(y_2_test, y_2_pred, squared=False)
print(f'SM2: R2 = {r2_2}, MAE = {mae_2}, RMSE = {rmse_2}')
cross_validate(sm2, X_2, y_2, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Elastic Net

In [ ]:
sm2 = ElasticNetCV()
sm2.fit(X_2_train, y_2_train)
y_2_pred = sm2.predict(X_2_test)
r2_2 = r2_score(y_2_test, y_2_pred)
mae_2 = mean_absolute_error(y_2_test, y_2_pred)
rmse_2 = mean_squared_error(y_2_test, y_2_pred, squared=False)
print(f'SM2: R2 = {r2_2}, MAE = {mae_2}, RMSE = {rmse_2}')
cross_validate(sm2, X_2, y_2, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### SGD

In [ ]:
sm2 = SGDRegressor()
sm2.fit(X_2_train, y_2_train)
y_2_pred = sm2.predict(X_2_test)
r2_2 = r2_score(y_2_test, y_2_pred)
mae_2 = mean_absolute_error(y_2_test, y_2_pred)
rmse_2 = mean_squared_error(y_2_test, y_2_pred, squared=False)
print(f'SM2: R2 = {r2_2}, MAE = {mae_2}, RMSE = {rmse_2}')
cross_validate(sm2, X_2, y_2, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### XGBoost

In [ ]:
sm2 = XGBRegressor(random_state=42, n_estimators=200, learning_rate=0.3, reg_lambda=0.5, reg_alpha=0.75)
%time sm2.fit(X_2_train, y_2_train)
y_2_pred = sm2.predict(X_2_test)
r2_2 = r2_score(y_2_test, y_2_pred)
mae_2 = mean_absolute_error(y_2_test, y_2_pred)
rmse_2 = mean_squared_error(y_2_test, y_2_pred, squared=False)
print(f'SM2: R2 = {r2_2}, MAE = {mae_2}, RMSE = {rmse_2}')
cross_validate(sm2, X_2, y_2, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

## Predicción SM3
Luego del análisis explorativo, se encontró una correlación entre las mediciones de SM3 y EV1, y entre SM3 y EV2.
Buscando aumentar el rendimiento del modelo, y analizando de mejor manera las relaciones entre los datos, se consideraron los dos sensores de EV2, el de salida de EV1, y el de SM2, lo cual aumentó en aproximadamente 15% el rendimiento del modelo.

In [ ]:
X_3 = df2[['SM2', 'EV1_2', 'EV2_1', 'EV2_2']]
y_3 = df2['SM3']
X_3_train, X_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, random_state=42, test_size=0.4)

### Regresión Linear

In [ ]:
sm3 = LinearRegression()
%time sm3.fit(X_3_train, y_3_train)
y_3_pred = sm3.predict(X_3_test) 
r2_3 = r2_score(y_3_test, y_3_pred)
mae_3 = mean_absolute_error(y_3_test, y_3_pred)
rmse_3 = mean_squared_error(y_3_test, y_3_pred, squared=False)
print(f'SM3: R2 = {r2_3}, MAE = {mae_3}, RMSE = {rmse_3}')
cross_validate(sm3, X_3, y_3, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Regresión Linear Polinomial

In [ ]:
poly = PolynomialFeatures(degree=4)
linear = LinearRegression()
%time sm3 = make_pipeline(poly, linear)
sm3.fit(X_3_train, y_3_train)
y_3_pred = sm3.predict(X_3_test)
r2_3 = r2_score(y_3_test, y_3_pred)
mae_3 = mean_absolute_error(y_3_test, y_3_pred)
rmse_3 = mean_squared_error(y_3_test, y_3_pred, squared=False)
print(f'SM3: R2 = {r2_3}, MAE = {mae_3}, RMSE = {rmse_3}')
cross_validate(sm3, X_3, y_3, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Regresión Lasso

In [ ]:
sm3 = LassoCV()
sm3.fit(X_3_train, y_3_train)
y_3_pred = sm3.predict(X_3_test)
r2_3 = r2_score(y_3_test, y_3_pred)
mae_3 = mean_absolute_error(y_3_test, y_3_pred)
rmse_3 = mean_squared_error(y_3_test, y_3_pred, squared=False)
print(f'SM3: R2 = {r2_3}, MAE = {mae_3}, RMSE = {rmse_3}')
cross_validate(sm3, X_3, y_3, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Elastic Net

In [ ]:
sm3 = ElasticNetCV()
sm3.fit(X_3_train, y_3_train)
y_3_pred = sm3.predict(X_3_test)
r2_3 = r2_score(y_3_test, y_3_pred)
mae_3 = mean_absolute_error(y_3_test, y_3_pred)
rmse_3 = mean_squared_error(y_3_test, y_3_pred, squared=False)
print(f'SM3: R2 = {r2_3}, MAE = {mae_3}, RMSE = {rmse_3}')
cross_validate(sm3, X_3, y_3, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### SGD

In [ ]:
sm3 = SGDRegressor()
sm3.fit(X_3_train, y_3_train)
y_3_pred = sm3.predict(X_3_test)
r2_3 = r2_score(y_3_test, y_3_pred)
mae_3 = mean_absolute_error(y_3_test, y_3_pred)
rmse_3 = mean_squared_error(y_3_test, y_3_pred, squared=False)
print(f'SM3: R2 = {r2_3}, MAE = {mae_3}, RMSE = {rmse_3}')
cross_validate(sm3, X_3, y_3, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### XGBoost

In [ ]:
sm3 = XGBRegressor(random_state=42, n_estimators=200, learning_rate=0.3, reg_lambda=0.5, reg_alpha=0.5)
sm3.fit(X_3_train, y_3_train)
y_3_pred = sm3.predict(X_3_test)
r2_3 = r2_score(y_3_test, y_3_pred)
mae_3 = mean_absolute_error(y_3_test, y_3_pred)
rmse_3 = mean_squared_error(y_3_test, y_3_pred, squared=False)
print(f'SM3: R2 = {r2_3}, MAE = {mae_3}, RMSE = {rmse_3}')
cross_validate(sm3, X_3, y_3, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

Observaciones:
El modelo arrogaba un puntaje de R2 de 89% aproximadamente. Se hizo un ajuste manual de hiperparámetros, lo que elevó la puntuación a ~95%. Para corroborar dichos valores, se hizo una búsqueda de hiperparámetros con validación cruzada, 

## Predicción SM4
Luego del análisis explorativo, se encontró una correlación entre las mediciones de SM4 y EV2, y entre SM4 y EDT1.

In [ ]:
X_4 = df2[['EV2_1','EV2_2', 'EDT%', 'EDT1']] 
y_4 = df2['SM4']
X_4_train, X_4_test, y_4_train, y_4_test = train_test_split(X_4, y_4, random_state=42, test_size=0.4)

### Regresión Linear

In [ ]:
sm4 = LinearRegression()
sm4.fit(X_4_train, y_4_train)
y_4_pred = sm4.predict(X_4_test)
r2_4 = r2_score(y_4_test, y_4_pred)
mae_4 = mean_absolute_error(y_4_test, y_4_pred)
rmse_4 = mean_squared_error(y_4_test, y_4_pred, squared=False)
print(f'SM4: R2 = {r2_4}, MAE = {mae_4}, RMSE = {rmse_4}')
cross_validate(sm4, X_4, y_4, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Regresión Lineal Polinomial

In [ ]:
poly = PolynomialFeatures(degree=4)
linear = LinearRegression()
sm4 = make_pipeline(poly, linear)
sm4.fit(X_4_train, y_4_train)
y_4_pred = sm4.predict(X_4_test)
r2_4 = r2_score(y_4_test, y_4_pred)
mae_4 = mean_absolute_error(y_4_test, y_4_pred)
rmse_4 = mean_squared_error(y_4_test, y_4_pred, squared=False)
print(f'SM4: R2 = {r2_4}, MAE = {mae_4}, RMSE = {rmse_4}')
cross_validate(sm4, X_4, y_4, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Regresión Lasso

In [ ]:
sm4 = LassoCV()
sm4.fit(X_4_train, y_4_train)
y_4_pred = sm4.predict(X_4_test)
r2_4 = r2_score(y_4_test, y_4_pred)
mae_4 = mean_absolute_error(y_4_test, y_4_pred)
rmse_4 = mean_squared_error(y_4_test, y_4_pred, squared=False)
print(f'SM4: R2 = {r2_4}, MAE = {mae_4}, RMSE = {rmse_4}')
cross_validate(sm4, X_4, y_4, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### Elastic Net

In [ ]:
sm4 = ElasticNetCV()
sm4.fit(X_4_train, y_4_train)
y_4_pred = sm4.predict(X_4_test)
r2_4 = r2_score(y_4_test, y_4_pred)
mae_4 = mean_absolute_error(y_4_test, y_4_pred)
rmse_4 = mean_squared_error(y_4_test, y_4_pred, squared=False)
print(f'SM4: R2 = {r2_4}, MAE = {mae_4}, RMSE = {rmse_4}')
cross_validate(sm4, X_4, y_4, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### SGD

In [ ]:
sm4 = SGDRegressor()
sm4.fit(X_4_train, y_4_train)
y_4_pred = sm4.predict(X_4_test)
r2_4 = r2_score(y_4_test, y_4_pred)
mae_4 = mean_absolute_error(y_4_test, y_4_pred)
rmse_4 = mean_squared_error(y_4_test, y_4_pred, squared=False)
print(f'SM4: R2 = {r2_4}, MAE = {mae_4}, RMSE = {rmse_4}')
cross_validate(sm4, X_4, y_4, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))

### XGBoost

In [ ]:
sm4 = XGBRegressor(random_state=42, n_estimators=175, learning_rate=0.302, reg_lambda=1, reg_alpha=0.5)
sm4.fit(X_4_train, y_4_train)
y_4_pred = sm4.predict(X_4_test)
r2_4 = r2_score(y_4_test, y_4_pred)
mae_4 = mean_absolute_error(y_4_test, y_4_pred)
rmse_4 = mean_squared_error(y_4_test, y_4_pred, squared=False)
print(f'SM4: R2 = {r2_4}, MAE = {mae_4}, RMSE = {rmse_4}')
cross_validate(sm4, X_4, y_4, cv=10, scoring=('r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'))